In [19]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2

import esim_torch
print("导入esim_torch")

import sys
sys.path.append('/home/gwp/rpg_vid2e/upsampling')  # 替换为实际的路径，这个路径包含了upsampling目录
from utils import Upsampler
print("导入Upsampler")


导入esim_torch
导入Upsampler


In [28]:
device = "cuda:0"
upsampler = Upsampler(input_dir="./example/original/seq0", output_dir="./example/upsampled/seq0")
upsampler.upsample()

# 读取原本图片
image_files = sorted(glob.glob("./example/original/seq0/imgs/*.png"))
images = np.stack([cv2.imread(f, cv2.IMREAD_GRAYSCALE) for f in image_files])

# 读取上采样后的图片
image_files = sorted(glob.glob("./example/upsampled/seq0/imgs/*.png"))
usampled_images = np.stack([cv2.imread(f, cv2.IMREAD_GRAYSCALE) for f in image_files])

print("原本图片的shape：", images.shape, "上采样后图片的shape：", usampled_images.shape)


Processing sequence number ./example/original/seq0


ImageSequence: 100%|██████████| 8/8 [00:11<00:00,  1.45s/it]

原本图片的shape： (9, 260, 346) 上采样后图片的shape： (65, 256, 320)


In [29]:
device = "cuda:0"
upsampler = Upsampler(input_dir="./esim_py/tests/data/images", output_dir="./esim_py/tests/data/upsampled/images") #注意，图像序列要放在文件夹imgs下，且需要有fps.txt记录帧率
upsampler.upsample()

print("Loading images")
# 读取原本图片
image_files = sorted(glob.glob("./esim_py/tests/data/images/images/*.png"))
images = np.stack([cv2.imread(f, cv2.IMREAD_GRAYSCALE) for f in image_files])

# 读取上采样后的图片
image_files = sorted(glob.glob("./esim_py/tests/data/upsampled/images/imgs/*.png"))
usampled_images = np.stack([cv2.imread(f, cv2.IMREAD_GRAYSCALE) for f in image_files])

print("原本图片的shape：", images.shape, "上采样后图片的shape：", usampled_images.shape)

Processing sequence number ./esim_py/tests/data/images


ImageSequence: 100%|██████████| 160/160 [00:26<00:00,  6.15it/s]


Loading images
原本图片的shape： (161, 180, 240) 上采样后图片的shape： (321, 160, 224)


In [42]:
t_refractory_period_ns=1e9;#某个时间段内不会产生event
esim = esim_torch.ESIM(contrast_threshold_neg=0.2, #定义负event的阈值
                        contrast_threshold_pos=0.2, #定义正event的阈值
                        refractory_period_ns=t_refractory_period_ns)  #refractory period in nanoseconds

print("从原始图像中生成事件")

# 读取时间戳
timestamps_s = np.genfromtxt("./esim_py/tests/data/images/timestamps.txt")
timestamps_ns = (timestamps_s * 1e9).astype("int64")

# 将图片转换为log scale
log_images = np.log(images.astype("float32") / 255 + 1e-4)

device = "cuda:0"
# torch tensor with type float32, shape T x H x W
log_images = torch.from_numpy(log_images).to(device)
# torch tensor with type int64,   shape T (已经转换为tensor)
timestamps_ns = torch.from_numpy(timestamps_ns).to(device)

# generate events with GPU support
print("Generating events")
events = esim.forward(log_images, timestamps_ns)

all_event = events
print(f"Total events: {all_event['x'].shape}")

从原始图像中生成事件
Generating events
Total events: torch.Size([19630])


In [43]:
print("从上采样的图像中生成事件")

esim_1 = esim_torch.ESIM(contrast_threshold_neg=0.2, #定义负event的阈值
                        contrast_threshold_pos=0.2, #定义正event的阈值
                        refractory_period_ns=t_refractory_period_ns)  #refractory period in nanoseconds

# 读取时间戳
upsample_timestamps_s = np.genfromtxt("./esim_py/tests/data/upsampled/timestamps.txt")
upsample_timestamps_ns = (upsample_timestamps_s * 1e9).astype("int64")

# 将图片转换为log scale
usampled_log_images = np.log(usampled_images.astype("float32") / 255 + 1e-4)

device = "cuda:0"
# torch tensor with type float32, shape T x H x W
usampled_log_images = torch.from_numpy(usampled_log_images).to(device)
# torch tensor with type int64,   shape T (已经转换为tensor)
upsample_timestamps_ns = torch.from_numpy(upsample_timestamps_ns).to(device)

# generate events with GPU support
print("Generating events")
usampled_events = esim_1.forward(usampled_log_images, upsample_timestamps_ns)

usampled_all_event = usampled_events
print(f"Total usampled events: {usampled_all_event['x'].shape}")

从上采样的图像中生成事件
Generating events
Total usampled events: torch.Size([18422])
